# Chapter 1 — The Python Data Model


## A Pythonic Card Deck


In [5]:
from french_deck import Card, FrenchDeck

In [5]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [6]:
deck = FrenchDeck()
len(deck)

52

In [7]:
deck[0]

Card(rank='2', suit='spades')

In [8]:
deck[-1]

Card(rank='A', suit='hearts')

In [9]:
from random import choice

choice(deck)

Card(rank='6', suit='clubs')

因为定义的 `__getitem__` 方法委托给了 `self._cards` 的 `[]` 操作符，所以 `deck` 对象自动支持切片。

In [10]:
# Top three cards
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [11]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

通常来说，特殊方法的调用都是隐式的，以 `for i in x` 这个语句会调用 `iter(x)` 方法，此方法实际上会调用 `x.__iter__()` ，当 `x.__iter__()` 未定义时，在 `FrenchDeck` 这个例子里会调用 `__getitem__` 。

所以，我们定义的 `__getitem__` 方法实际上让 `FrenchDeck` 可迭代了。

In [12]:
for card in deck:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

In [13]:
for card in reversed(deck):
    print(card)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

迭代通常是隐式的，如果一个集合没有定义 `__contains__` 方法，`in` 操作符会对集合执行顺序的扫描。

In [14]:
Card('Q', 'hearts') in deck

True

In [15]:
Card('X', 'hearts') in deck


False

实际上，我们的代码应该尽量减少特殊方法的直接调用，大多数情况下，我们应该使用相关的内建方法来触发相应特殊方法的调用。

比如使用 `len` 就是去触发 `__len__` 方法。

## Emulating Numeric Types

### String Representation

In [16]:
from vector2d import Vector

In [17]:
# string representation
v = Vector(3, 4)
v

Vector(3, 4)

对于未定义 `__repr__` 的对象：

In [18]:
class V:
    def __init__(self, x: int, y: int) -> None:
        self.x: int = x
        self.y: int = y
        
v = V(3, 4)
v

对于 `__repr__` 和 `__str__` 方法的区别，stack overflow 上的总结是:

* `__repr__` 的目标是准确性
* `__str__` 的目标是可读性
* 容器的 `__str__` 方法使用了容器元素的 `__repr__` 方法。

更多信息可见： [What is the difference between __str__ and __repr__?](https://stackoverflow.com/questions/1436703/what-is-the-difference-between-str-and-repr)

一般来说，我们只需要定义 `__repr__` 方法，此方法的定义输出要和源码保持一致，方便我们通过输出重建此对象（`eval` 方法）来进行 debug ，所以我们 `f-string` 中的 `!r` 表示法才显得重要。

因为 `print` 方法会打印 `__str__` 的对象表示法，某些情况下我们再多定义 `__str__` 方法来提高打印的可读性。

In [19]:
class V:
    def __init__(self, x: int, y: int) -> None:
        self.x: int = x
        self.y: int = y
    
    def __repr__(self) -> str:
        return f'V({self.x!r}, {self.y!r})'
    
    def __str__(self) -> str:
        return f'V(x = {self.x}, y = {self.y})'

v = V(3, 4)
v

V(3, 4)

In [20]:
v = eval('V(3, 4)')
v.x

3

In [21]:
print(v)

V(x = 3, y = 4)


### Boolean Value of a Custom Type

在未定义 `__bool__` 和 `__len__` 的情况下, 用户自定义类的实例被认为是 `true`。

当定义了 `__bool__` 和 `__len__` 的情况下，`bool(x)` 表达式会优先调用 `x.__bool__()` 作为结果。

当未定义 `__bool__` 但是 定义了 `__len__` 的情况下，`x.__len__()` 的返回值等于 0 时，`bool(x)` 等于 `False`，否则返回 `True`。

In [22]:
# user-defined class without `__bool__` and `__len__` methods.
class T:
    def __init__(self) -> None:
        ...

t = T()
bool(t)

True

In [23]:
# user-defined class with `__bool__` method.
class T:
    def __init__(self, x: int) -> None:
        self.x = x
        
    def __bool__(self) -> bool:
        return self.x != 0

t = T(0)
bool(t)

False

In [24]:
# user-defined class with `__len__` method.
class T:
    def __init__(self) -> None:
        self.items: list = []
        
    def __len__(self) -> int:
        return len(self.items)
t = T()
bool(t)

False